# OpenStreetMap Project
# Data Wrangling with MongoDB
### by Dogan Askan                                                
### Map Area: Syracuse, NY, United States

### Part 1 - Data Tidying

In [26]:
#We can start investigating the data by counting tags
import xml.etree.cElementTree as ET
import pprint
filename="syracuse_new-york.osm"
def count_tags(filename):
    dic={}
    for event, elem in ET.iterparse(filename):
        if elem.tag not in dic.keys():
            dic[elem.tag]=1
        else:
            dic[elem.tag]+=1
    return dic

print count_tags(filename)

{'node': 279475, 'nd': 328321, 'bounds': 1, 'member': 5549, 'tag': 215590, 'relation': 793, 'way': 35493, 'osm': 1}


In [3]:
#It seems that there are 194 unique user id in dataset, we will recheck after importing with a query 
import xml.etree.cElementTree as ET
import pprint
import re

filename="syracuse_new-york.osm"

def process_map(filename):
    """It is for checking the unique users
    """
    users = set()
    for _, element in ET.iterparse(filename):
        if element.tag == "node":
            ta = element.attrib['uid']
            users.add(ta)
        pass
    return users

users = process_map(filename)
pprint.pprint(len(users))

194


In [120]:
#Here, we can check problematic fields such as "postcode" and "streetname".
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

filename="syracuse_new-york.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Highway", "Way"]

mapping = { "St": "Street","St.": "Street","Ave": "Avenue","Rd.": "Road","Rd": "Road"}

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if element.tag == "tag":
            if element.attrib['k']=="addr:street":
                namelist=element.attrib['v'].split(" ")
                if namelist[-1] not in expected:
                    users.add(namelist[-1])
    return users

def process_map_zip(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if element.tag == "tag":
            if element.attrib['k']=="addr:postcode":
                zipcode=element.attrib['v']
                if len(zipcode)!=5:
                    users.add(zipcode)
    return users

#audit(filename)
pprint.pprint(process_map(filename))
pprint.pprint(process_map_zip(filename))

set(['11',
     '298',
     '31',
     'Center',
     'Circle',
     'Courts',
     'East',
     'Path',
     'Plaza',
     'Rowe',
     'Run',
     'St',
     'Terrace',
     'Turnpike',
     'West'])
set(['13202-1107',
     '13204-1243',
     '132059211',
     '13206-2238',
     '13210-1053',
     '13210-1203',
     '13214-1303',
     '132179211',
     '13218-1185',
     '13219-331',
     '13224-1110'])


As seen above, there are a few unsual street name endings. We can programmatically update some such as "St" but there are also things that we cannot update programmatically such as "11", "298" and "31". We can write a code to update it into "Street".

And, for zip codes, there are some codes longer than 5 digits. Those are technically correct. However we can write a code for consistency to just get the first 5 digits.

In [139]:
#To check the latitude and longitude ranges if those are valid
#According to Google Maps limits for Syracuse are (42.98,43.09) in latitude and (-76.21,-76.07) in longitude
def process_map_lat(filename):
    lats = []
    i=0
    for _, element in ET.iterparse(filename):
        if element.tag == "node":
            i+=1
            if float(element.attrib['lat'])<42.98 or float(element.attrib['lat'])>43.09:
                lats.append(element.attrib['lat'])
    return lats
lats=process_map_lat(filename)
print "Percent of out of range latitudes: "+str(round((len(lats)/float(i))*100,2))+"%"
print "with min latitude "+str(min(lats))+" and max latitude "+str(max(lats))

def process_map_lon(filename):
    lons = []
    i=0
    for _, element in ET.iterparse(filename):
        if element.tag == "node":
            i+=1
            if float(element.attrib['lon'])<-76.21 or float(element.attrib['lon'])>-76.07:
                lons.append(element.attrib['lon'])
    return lons
lons=process_map_lon(filename)
print "Percent of out of range longitudes: "+str(round((len(lons)/float(i))*100,2))+"%"
print "with min longitude "+str(max(lons))+" and max longitude "+str(min(lons))

Percent of out of range latitudes: 27.14%
with min latitude 42.941 and max latitude 43.2189814
Percent of out of range longitudes: 36.91%
with min longitude -76.375 and max longitude -75.9460041


Altough out of range ratios look huge, we can still say that max and min values are very close to city center. So, we can ignore these and there is no need to create a function for that.

In [5]:
#This code turns .osm into .json by making it more understandable. 
#And, two functions (update_name and update_zip) update street names and zip codes as we discussed earlier.
import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]
mapping = { "St": "Street","St.": "Street","Ave": "Avenue","Rd.": "Road","Rd": "Road"}
def update_name(name, mapping): #to update street names
    namelist=name.split(" ")
    if namelist[-1] in mapping.keys():
        namelist[-1]=mapping[namelist[-1]]
        name=' '.join(namelist)
    return name

def update_zip(zipcode): #to update zip codes
    if len(zipcode)!=5:
        zipcode=zipcode[:5]
    return zipcode

def shape_element(element):
    """This is for updating .osm elements to nice .json dictionary"""
    node = {}
    address={}
    created={}
    ad="addr:"
    pos=[]
    node_refs=[]
    if element.tag == "node" or element.tag == "way" :
        """Part 1 - For type key"""
        if element.tag == "node":
            node['type']='node'
        else:
            node['type']='way'    
        """Part 2 - For address key"""
        for i in element.iter('tag'):
            if ad in i.attrib['k']:
                if ":" in i.attrib['k'].replace(ad,""):
                    pass
                else:
                    if i.attrib['k'].replace(ad,"")=="street":
                        address[i.attrib['k'].replace(ad,"")]=update_name(i.attrib['v'],mapping)
                    elif i.attrib['k'].replace(ad,"")=="postcode":
                        address[i.attrib['k'].replace(ad,"")]=update_zip(i.attrib['v'])
                    else:
                        address[i.attrib['k'].replace(ad,"")]=i.attrib['v']
            else:    
                node[i.attrib['k']]=i.attrib['v']
        if address:
            node["address"]=address
        """Part 3 - For pos and created keys"""
        for i in element.iter('node'):
            for key, value in i.attrib.items():
                if key in CREATED:
                    created[key]=value
                    k=1
                elif key=="lat" or key=="lon":
                    pos.append(float(value))     
                else:
                    node[key]=value
        if created:
            node['created']=created
        if pos:
            node['pos']=pos
        """Part 4 - For node_refs key"""        
        for i in element.iter('nd'):
            node_refs.append(i.attrib['ref'])
            l=1
        if node_refs:
            node['node_refs']=node_refs       
        return node
    else:
        return None

def process_map(file_in, pretty = False):
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

output=process_map("syracuse_new-york.osm", pretty = False)

### Part 2 - Data Overview

In this part, we can analyze the date by using MongoDB

File sizes                                              
syracuse_new-york.osm ........... 61.2 MB  
syracuse_new-york.osm.json .... 63.7 MB

In [7]:
import pprint
from pymongo import MongoClient
client = MongoClient('localhost:27017')
db = client.examples
pprint.pprint(db.syracuse.find().count()) #Number of documents

314968


In [65]:
pprint.pprint(db.syracuse.find({"type":"node"}).count()) #node count

279473


In [69]:
pprint.pprint(db.syracuse.find({"type":"way"}).count()) #way count

35476


Number of unique users
> db.syracuse.distinct("created.user").length  
194 

In [15]:
#Top 1 contributing user
def make_pipeline():
    pipeline = [{"$group":{"_id":"$created.user", "count":{"$sum":1}}}, {"$sort":{"count":-1}}, {"$limit":1}]
    return pipeline

def syracuse(db, pipeline):
    return [doc for doc in db.syracuse.aggregate(pipeline)]

if __name__ == '__main__':
    result = syracuse(db, make_pipeline())
    pprint.pprint(result[:5])

[{u'_id': u'zeromap', u'count': 136837}]


In [13]:
#Number of users appearing only once (having 1 post)
def make_pipeline():
    pipeline = [{"$group":{"_id":"$created.user", "count":{"$sum":1}}}, {"$group":{"_id":"$count", "num_users":{"$sum":1}}}, 
                {"$sort":{"_id":1}}, {"$limit":1}]
    return pipeline

def syracuse(db, pipeline):
    return [doc for doc in db.syracuse.aggregate(pipeline)]

if __name__ == '__main__':
    result = syracuse(db, make_pipeline())
    pprint.pprint(result[:5])

[{u'_id': 1, u'num_users': 45}]


In [88]:
#Here, we can see top 5 zip codes
def make_pipeline():
    pipeline = [{"$match":{"address.postcode":{"$exists":1}}}, 
                   {"$group":{"_id":"$address.postcode", "count":{"$sum":1}}}, 
                {"$sort":{"count":-1}}]
    return pipeline

def syracuse(db, pipeline):
    return [doc for doc in db.syracuse.aggregate(pipeline)]

if __name__ == '__main__':
    result = syracuse(db, make_pipeline())
    pprint.pprint(result[:5])

[{u'_id': u'13224', u'count': 821},
 {u'_id': u'13214', u'count': 514},
 {u'_id': u'13210', u'count': 475},
 {u'_id': u'13206', u'count': 290},
 {u'_id': u'13205', u'count': 284}]


In [89]:
#Here, we can see top 5 street names
def make_pipeline():
    pipeline = [{"$match":{"address.street":{"$exists":1}}}, 
                   {"$group":{"_id":"$address.street", "count":{"$sum":1}}}, 
                {"$sort":{"count":-1}}]
    return pipeline

def syracuse(db, pipeline):
    return [doc for doc in db.syracuse.aggregate(pipeline)]

if __name__ == '__main__':
    result = syracuse(db, make_pipeline())
    pprint.pprint(result[:5])

[{u'_id': u'Erie Boulevard East', u'count': 263},
 {u'_id': u'Westmoreland Avenue', u'count': 206},
 {u'_id': u'South Salina Street', u'count': 197},
 {u'_id': u'East Genesee Street', u'count': 163},
 {u'_id': u'Westcott Street', u'count': 155}]


### Part 3 - Additional Ideas

Consequently, this map is incomplete and there many things that can be done such as better street name updates or adding additional a field for last 4 digits of zip codes. Yet again, these improvement will definitely be beneficial to provide more efficient, explainable and faster database quries. Regarding the coordinates, I don't think it is a problem because the closest city to those points is definitely Syracuse.

In [100]:
#Here, we can see top 10 catagories.
def make_pipeline():
    pipeline = [{"$match":{"amenity":{"$exists":1}}}, 
                   {"$group":{"_id":"$amenity", "count":{"$sum":1}}}, {"$sort":{"count":-1}}]
    return pipeline

def syracuse(db, pipeline):
    return [doc for doc in db.syracuse.aggregate(pipeline)]

if __name__ == '__main__':
    result = syracuse(db, make_pipeline())
    pprint.pprint(result[:10])

[{u'_id': u'parking', u'count': 939},
 {u'_id': u'school', u'count': 186},
 {u'_id': u'fast_food', u'count': 158},
 {u'_id': u'bench', u'count': 153},
 {u'_id': u'restaurant', u'count': 152},
 {u'_id': u'place_of_worship', u'count': 131},
 {u'_id': u'fuel', u'count': 122},
 {u'_id': u'bank', u'count': 64},
 {u'_id': u'post_box', u'count': 57},
 {u'_id': u'pharmacy', u'count': 51}]


In [106]:
#Amenity ratios in Syracuse
total=db.syracuse.find({"amenity":{"$exists":1}}).count() #All amenties
def make_pipeline():
    pipeline = [{"$match":{"amenity":{"$exists":1}}},{"$group":{"_id":"$amenity", "count":{"$sum":1}}}, 
                {"$project":{"_id":"$_id","ratio":{"$divide":["$count",total/100]}}},
                {"$sort":{"ratio":-1}}]
    return pipeline

def syracuse(db, pipeline):
    return [doc for doc in db.syracuse.aggregate(pipeline)]

if __name__ == '__main__':
    result = syracuse(db, make_pipeline())
    pprint.pprint(result[:10])

[{u'_id': u'parking', u'ratio': 34.77777777777778},
 {u'_id': u'school', u'ratio': 6.888888888888889},
 {u'_id': u'fast_food', u'ratio': 5.851851851851852},
 {u'_id': u'bench', u'ratio': 5.666666666666667},
 {u'_id': u'restaurant', u'ratio': 5.62962962962963},
 {u'_id': u'place_of_worship', u'ratio': 4.851851851851852},
 {u'_id': u'fuel', u'ratio': 4.518518518518518},
 {u'_id': u'bank', u'ratio': 2.3703703703703702},
 {u'_id': u'post_box', u'ratio': 2.111111111111111},
 {u'_id': u'pharmacy', u'ratio': 1.8888888888888888}]


Why are there that much "parking"? It may be some mistake needs to be analyzed further.

In [93]:
#Top 10 fast food places, Subway wins!
def make_pipeline():
    pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity":"fast_food"}}, 
                   {"$group":{"_id":"$name", "count":{"$sum":1}}}, {"$sort":{"count":-1}}]
    return pipeline

def syracuse(db, pipeline):
    return [doc for doc in db.syracuse.aggregate(pipeline)]

if __name__ == '__main__':
    result = syracuse(db, make_pipeline())
    pprint.pprint(result[:10])

[{u'_id': u'Subway', u'count': 18},
 {u'_id': u"McDonald's", u'count': 15},
 {u'_id': u"Dunkin' Donuts", u'count': 12},
 {u'_id': u'Burger King', u'count': 11},
 {u'_id': u"Wendy's", u'count': 5},
 {u'_id': u'Original Italian Pizza', u'count': 4},
 {u'_id': u'Taco Bell', u'count': 4},
 {u'_id': u"Arby's", u'count': 3},
 {u'_id': u"Pavone's Pizza", u'count': 3},
 {u'_id': u'KFC', u'count': 3}]


There is nothing unsual.

In [16]:
#Let's check contribution ratios
total=db.syracuse.find({"created.user":{"$exists":1}}).count() #All users
def make_pipeline():
    pipeline = [{"$match":{"created.user":{"$exists":1}}},{"$group":{"_id":"$created.user", "count":{"$sum":1}}}, 
                {"$project":{"_id":"$_id","ratio":{"$divide":["$count",total/100]}}},
                {"$sort":{"ratio":-1}}]
    return pipeline

def syracuse(db, pipeline):
    return [doc for doc in db.syracuse.aggregate(pipeline)]

if __name__ == '__main__':
    result = syracuse(db, make_pipeline())
    pprint.pprint(result[:10])

[{u'_id': u'zeromap', u'ratio': 48.97530422333572},
 {u'_id': u'woodpeck_fixbot', u'ratio': 27.032211882605583},
 {u'_id': u'DTHG', u'ratio': 8.641016463851109},
 {u'_id': u'RussNelson', u'ratio': 2.6904080171796707},
 {u'_id': u'yhahn', u'ratio': 2.48067287043665},
 {u'_id': u'fx99', u'ratio': 1.473514674302076},
 {u'_id': u'timr', u'ratio': 0.9659985683607731},
 {u'_id': u'TIGERcnl', u'ratio': 0.7415891195418755},
 {u'_id': u'Johnc', u'ratio': 0.6717967072297781},
 {u'_id': u'ECRock', u'ratio': 0.5662133142448104}]


The first user has almost half of the total contributions. And, top 5 users have the 90%. More active contributors are needed to improve the map. Currently, it looks that it is only actively contributed by a few users. I am sure there are many smart phone and online map users never heard OpenStreetMap yet. I, personally, wasn't aware of it before this course. So, better advertising is essential to provide more active contributors. By doing so, the first problem "small data" can be fixed.  

In addition to this, data coming from contributors need to be clean and consistent. To do so, for example, OpenStreetMap can limit the zip code attribute as only 5 digits. Or, a recommender system can be utilized for some section to recommend "Street" or "Road" while "St" or "Rd" are being written.